# Substitution

Mathematical complexity: 256!

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import math

def monoalphabetic_key_generation():
    original_pool = np.arange(0,256)
    shuffled_pool = np.copy(original_pool)
    np.random.shuffle(shuffled_pool)
    mapping = dict(zip(original_pool, shuffled_pool))
    reverse_mapping = dict(zip(shuffled_pool, original_pool))
    return mapping,reverse_mapping

def monoalphabetic_cipher_color_encrypt(image,mapping_key):
    key_list = np.array(list(mapping_key.keys()))
    value_list = np.array(list(mapping_key.values()))
    mapping_array = np.zeros(key_list.max()+1,dtype=value_list.dtype)
    mapping_array[key_list] = value_list
    encrypted_image = mapping_array[image]
    return encrypted_image

def monoalphabetic_cipher_color_decrypt(image,reverse_mapping_key):
    key_list = np.array(list(reverse_mapping_key.keys()))
    value_list = np.array(list(reverse_mapping_key.values()))
    reverse_mapping_array = np.zeros(key_list.max()+1,dtype=value_list.dtype)
    reverse_mapping_array[key_list] = value_list
    decrypted_image = reverse_mapping_array[image]
    return decrypted_image

key,reverse_key = monoalphabetic_key_generation()
# key = {1:255,5:234,2:231,7:45,47:78,249:113}
# reverse_key = {255:1,234:5,231:2,45:7,78:47,113:249}
img = cv2.imread('Sample.png')
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

encrypted_img = monoalphabetic_cipher_color_encrypt(img, key)
plt.imsave("Sample_encrypted_substitution.png",encrypted_img.astype('uint8'))
decrypted_img = monoalphabetic_cipher_color_decrypt(encrypted_img, reverse_key)
plt.imsave("Sample_decrypted_substitution.png",decrypted_img.astype('uint8'))

In [2]:
start = time.time()
encrypted_img = monoalphabetic_cipher_color_encrypt(img, key)
end = time.time()
encryption_time_substitution = end-start
print("Encryption time (Substitution): "+str(end-start))

Encryption time (Substitution): 0.025204896926879883


In [3]:
start = time.time()
decrypted_img = monoalphabetic_cipher_color_decrypt(encrypted_img, reverse_key)
end = time.time()
decryption_time_substitution = end-start
print("Decryption time (Substitution): "+str(end-start))

Decryption time (Substitution): 0.03002142906188965


# AES
Mathematical complexity: 2^256

In [4]:
import time

In [5]:
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad, unpad
from Cryptodome.Random import get_random_bytes

# specify the key and initialization vector (IV)
key = key = get_random_bytes(32)
iv = iv = get_random_bytes(16)

# create the AES cipher object
cipher = AES.new(key, AES.MODE_CBC, iv)

# read the image data
with open('Sample.png', 'rb') as f:
    data = f.read()

# pad the data to a multiple of 16 bytes using PKCS7 padding
padded_data = pad(data, AES.block_size, style='pkcs7')

# encrypt the data
encrypted_data = cipher.encrypt(padded_data)

# write the encrypted data to a file
with open('Sample_encrypted_AES.png', 'wb') as f:
    f.write(encrypted_data)

# decrypt the encrypted data
cipher = AES.new(key, AES.MODE_CBC, iv)
decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size, style='pkcs7')

# write the decrypted data to a file
with open('Sample_decrypted_AES.png', 'wb') as f:
    f.write(decrypted_data)

In [6]:
cipher = AES.new(key, AES.MODE_CBC, iv)
start = time.time()
with open('Sample.png', 'rb') as f:
    data = f.read()
padded_data = pad(data, AES.block_size, style='pkcs7')
encrypted_data = cipher.encrypt(padded_data)
with open('Sample_encrypted_AES.png', 'wb') as f:
    f.write(encrypted_data)
end = time.time()
encryption_time_AES = end-start
print("Encryption time (AES): "+str(end-start))

Encryption time (AES): 0.015279769897460938


In [7]:
cipher = AES.new(key, AES.MODE_CBC, iv)
start = time.time()
with open('Sample_encrypted_AES.png', 'rb') as f:
    encrypted_data = f.read()
decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size, style='pkcs7')
with open('Sample_decrypted_AES.png', 'wb') as f:
    f.write(decrypted_data)
end = time.time()
decryption_time_AES = end-start
print("Decryption time (AES): "+str(end-start))

Decryption time (AES): 0.03503084182739258


# RC4
Mathematical complexity: 2^80

In [8]:
def rc4_encrypt(data, key):
    S = list(range(256))
    j = 0
    out = []

    # Key-scheduling algorithm
    for i in range(256):
        j = (j + S[i] + key[i % len(key)]) % 256
        S[i], S[j] = S[j], S[i]

    # Pseudo-random generation algorithm
    i = j = 0
    for byte in data:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        out.append(byte ^ S[(S[i] + S[j]) % 256])

    return bytes(out)


def rc4_decrypt(data, key):
    return rc4_encrypt(data, key)

# Load image data
with open("Sample.png", "rb") as f:
    data = f.read()

# Encrypt image data with RC4
key = b"my-secret-key"
encrypted_data = rc4_encrypt(data, key)

with open("Sample_encrypted_RC4.png", "wb") as f:
    f.write(encrypted_data)

# Decrypt encrypted image data with RC4
decrypted_data = rc4_decrypt(encrypted_data, key)

# Save decrypted image data to file
with open("Sample_decrypted_RC4.png", "wb") as f:
    f.write(decrypted_data)

In [9]:
start = time.time()
with open('Sample.png', 'rb') as f:
    data = f.read()
encrypted_data = rc4_encrypt(data, key)
with open("Sample_encrypted_RC4.png", "wb") as f:
    f.write(encrypted_data)
end = time.time()
encryption_time_RC4 = end-start
print("Encryption time (RC4): "+str(end-start))

Encryption time (RC4): 0.8651797771453857


In [10]:
start = time.time()
with open('Sample_encrypted_RC4.png', 'rb') as f:
    encrypted_data = f.read()
decrypted_data = rc4_decrypt(encrypted_data, key)
with open("Sample_decrypted_RC4.png", "wb") as f:
    f.write(decrypted_data)
end = time.time()
decryption_time_RC4 = end-start
print("Decryption time (RC4): "+str(end-start))

Decryption time (RC4): 0.8865010738372803


# ChaCha20
Mathematical Complexity: 2^352

In [11]:
from Cryptodome.Cipher import ChaCha20
from Cryptodome.Random import get_random_bytes
from PIL import Image
import io

# Generate a random key and nonce
key = get_random_bytes(32)
nonce = get_random_bytes(8)

# Open the image file
with open("Sample.png", "rb") as f:
    img_bytes = f.read()

# Create a new ChaCha20 cipher object
cipher = ChaCha20.new(key=key, nonce=nonce)

# Encrypt the image data
encrypted_img_bytes = cipher.encrypt(img_bytes)
with open("Sample_encrypted_ChaCha20.png", "wb") as f:
    f.write(encrypted_img_bytes)

# Decrypt the encrypted image data
cipher = ChaCha20.new(key=key, nonce=nonce)
decrypted_img_bytes = cipher.decrypt(encrypted_img_bytes)

with open("Sample_decrypted_ChaCha20.png", "wb") as f:
    f.write(decrypted_img_bytes)

In [12]:
cipher = ChaCha20.new(key=key, nonce=nonce)
start = time.time()
with open("Sample.png", "rb") as f:
    img_bytes = f.read()
encrypted_img_bytes = cipher.encrypt(img_bytes)
with open("Sample_encrypted_ChaCha20.png", "wb") as f:
    f.write(encrypted_img_bytes)
end = time.time()
encryption_time_ChaCha20 = end-start
print("Encryption time (ChaCha20): "+str(end-start))

Encryption time (ChaCha20): 0.014959573745727539


In [13]:
cipher = ChaCha20.new(key=key, nonce=nonce)
start = time.time()
with open("Sample_encrypted_ChaCha20.png", "rb") as f:
    encrypted_img_bytes = f.read()
decrypted_img_bytes = cipher.decrypt(encrypted_img_bytes)
with open("Sample_decrypted_ChaCha20.png", "wb") as f:
    f.write(decrypted_img_bytes)
end = time.time()
decryption_time_ChaCha20 = end-start
print("Decryption time (ChaCha20): "+str(end-start))

Decryption time (ChaCha20): 0.03516054153442383
